In [1]:
from sklearn import datasets
from sklearn.model_selection import KFold

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm


class SimpleMLP(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.fc_in = nn.Linear(in_features=4, out_features=5)
        self.fc2 = nn.Linear(in_features=5, out_features=7)
        self.fc_out = nn.Linear(in_features=7, out_features=n_classes)

    def forward(self, x):
        x = self.fc_in(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc_out(x)
        x = F.softmax(x, dim=1)
        return x


def train(
    model,
    train_loader,
    valid_loader,
    optimizer,
    criterion,
    device,
    epochs=10,
    log_writer=None,
):

    history = {
        "train": {"loss": [], "accuracy": []},
        "valid": {"loss": [], "accuracy": []},
    }
    n = len(train_loader)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        Loss_epoch = 0
        correct = 0
        total = 0
        evaluation_train = {"accuracy": 0, "loss": 0}
        for idx, data in enumerate(tqdm(train_loader)):
            input, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            output = model(input)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            Loss_epoch += loss.item() * len(labels)
            correct += accuracy(output, labels) * len(labels)
            total += len(labels)
        # Loss_history.append(Loss_epoch/n)
        evaluation_train["accuracy"] = correct / total
        evaluation_train["loss"] = Loss_epoch / total
        # evaluation_train = evaluate(model, train_loader,criterion, device)
        evaluation_valid, _ = evaluate(model, valid_loader, criterion, device)
        print(f"epoch: {epoch}, train: {evaluation_train}, valid: {evaluation_valid}")

        history["train"]["accuracy"].append(evaluation_train["accuracy"])
        history["train"]["loss"].append(evaluation_train["loss"])
        history["valid"]["loss"].append(evaluation_valid["loss"])
        history["valid"]["accuracy"].append(evaluation_valid["accuracy"])

        if log_writer is not None:
            log_writer.log(
                {
                    "train/train_loss": evaluation_train["loss"],
                    "train/train_accuracy": evaluation_train["accuracy"],
                    "val/valid_loss": evaluation_valid["loss"],
                    "val/valid_accuracy": evaluation_valid["accuracy"],
                }
            )

    return history


def accuracy(output, labels):
    _, preds = torch.max(output, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds)).item()


def evaluate(model, data_loader, criterion, device, return_preds=False):
    model.eval()
    Accuracy_history = []
    Loss_history = []
    PREDS = []
    with torch.no_grad():
        for idx, data in enumerate(data_loader):
            input, target = data[0].to(device), data[1].to(device)
            output = model(input)
            loss = criterion(output, target)
            Accuracy_history.append(accuracy(output, target))
            Loss_history.append(loss.item())
            if return_preds:
                # PREDS.extend(torch.max(output, dim=1)[1].tolist())
                PREDS.extend(output.tolist())

    return {
        "accuracy": torch.mean(torch.Tensor(Accuracy_history)).item(),
        "loss": torch.mean(torch.Tensor(Loss_history)).item(),
    }, PREDS

In [29]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2 as tt
from sklearn.model_selection import train_test_split


class IRISDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x  # torch.Tensor(x,).unsqueeze(1).type(torch.FloatTensor)
        self.y = y  # torch.Tensor(y).type(torch.LongTensor)
        self.transform = transform

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.x)

In [30]:
iris_X, iris_Y = datasets.load_iris(return_X_y=True)
iris_X = iris_X.astype("float32")
iris_Y = iris_Y.astype("int64")

In [33]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from torch.utils.tensorboard import SummaryWriter

class custom_summary_writer:
    def __init__(self, writer):
        self.writer = writer

    def log(self, metrics):
        for key, value in metrics.items():
            self.writer.add_scalar(key, value)
            
writer = custom_summary_writer(SummaryWriter("runs/iris_experiment_1"))

criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pipeline = make_pipeline(StandardScaler())

kFold = KFold(n_splits=10, random_state=42, shuffle=True)
for fold, (train_index, test_index) in enumerate(kFold.split(iris_X)):
    print(f"Kfold: {fold}")
    X_train, X_test, y_train, y_test = (
        iris_X[train_index],
        iris_X[test_index],
        iris_Y[train_index],
        iris_Y[test_index],
    )

    X_train = pipeline.fit_transform(X_train)
    X_test = pipeline.transform(X_test)

    X_train_dataset = IRISDataset(X_train, y_train, tt.Compose([torch.tensor]))
    X_test_dataset = IRISDataset(X_test, y_test, tt.Compose([torch.tensor]))

    train_loader = DataLoader(X_train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(X_test_dataset, batch_size=10, shuffle=False)

    model = SimpleMLP(3)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    history = train(
        model, train_loader, test_loader, optimizer, criterion, device, epochs=1,log_writer = writer
    )

Kfold: 0


100%|██████████| 14/14 [00:00<00:00, 231.78it/s]


epoch: 0, train: {'accuracy': 0.3481481538878547, 'loss': 1.1214772286238495}, valid: {'accuracy': 0.30000001192092896, 'loss': 1.135094165802002}
Kfold: 1


100%|██████████| 14/14 [00:00<00:00, 295.73it/s]


epoch: 0, train: {'accuracy': 0.2814814895391464, 'loss': 1.0993227782072845}, valid: {'accuracy': 0.15000000596046448, 'loss': 1.083238124847412}
Kfold: 2


...

epoch: 0, train: {'accuracy': 0.2666666722959942, 'loss': 1.111108417864199}, valid: {'accuracy': 0.10000000149011612, 'loss': 1.1189820766448975}
Kfold: 8


100%|██████████| 14/14 [00:00<00:00, 249.30it/s]


epoch: 0, train: {'accuracy': 0.3333333377484922, 'loss': 1.1049202548133001}, valid: {'accuracy': 0.25, 'loss': 1.126150369644165}
Kfold: 9


100%|██████████| 14/14 [00:00<00:00, 222.38it/s]

epoch: 0, train: {'accuracy': 0.3555555608537462, 'loss': 1.0894309282302856}, valid: {'accuracy': 0.10000000149011612, 'loss': 1.1283395290374756}
